In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import StringType, TimestampType
from delta.tables import DeltaTable

TABLE_SOURCE = "v_credit.bronze.pesquisa_satisfacao"
TABLE_TARGET_VALIDOS = "v_credit.silver.tb_pesquisa"
TABLE_TARGET_INVALIDOS = "v_credit.silver.tb_pesquisa_invalidos"

In [0]:
from pyspark.sql import functions as F

# 1. Leitura da Tabela Bronze
df_bronze = spark.table(TABLE_SOURCE)

# 2. Identificar a Carga Mais Recente
# Usa 'ingestion_timestamp' para encontrar o timestamp máximo da última carga.
max_dt_ingestion = (
    df_bronze
    .agg(F.max(F.col("ingestion_timestamp")).alias("max_ts"))
    .first()["max_ts"]
)

# 3. Filtrar o DataFrame Bronze apenas pela carga mais recente
df_filtrado = df_bronze.filter(
    F.col("ingestion_timestamp") == F.lit(max_dt_ingestion)
)

# 4. Aplicação das Transformações (usando df_filtrado)
df_transformado = (
    df_filtrado # <--- USANDO O DATAFRAME FILTRADO AQUI
    .select(
        F.col("id_pesquisa").cast("bigint").alias("cd_pesquisa"),
        F.col("id_chamado").cast("bigint").alias("cd_chamado"),
        F.when(
            (F.col("nota_atendimento").isNull()) | 
            (F.trim(F.col("nota_atendimento")) == ""),
            F.lit(None)
        ).otherwise(
            F.col("nota_atendimento").cast("smallint")
        ).alias("nu_nota"),
        F.col("ingestion_timestamp").alias("dt_ingestion"),
        F.coalesce(F.col("origem"), F.lit("sistema_pesquisa")).alias("dc_origem")
    )
    .dropDuplicates(["cd_pesquisa"])
)

# O restante do seu pipeline de validação e separação permanece o mesmo

df_validacao = (
    df_transformado
    .withColumn("flag_pk_valida", F.col("cd_pesquisa").isNotNull())
    .withColumn("flag_fk_valida", F.col("cd_chamado").isNotNull())
    .withColumn(
        "flag_nota_valida",
        F.col("nu_nota").isNull() | 
        ((F.col("nu_nota") >= 1) & (F.col("nu_nota") <= 5))
    )
    .withColumn(
        "flag_qualidade",
        F.when(
            F.col("flag_pk_valida") &
            F.col("flag_fk_valida") &
            F.col("flag_nota_valida"),
            F.lit("OK")
        ).otherwise(F.lit("ERRO"))
    )
)

df_silver = (
    df_validacao
    .filter(F.col("flag_qualidade") == "OK")
    .drop("flag_pk_valida", "flag_fk_valida", "flag_nota_valida", "flag_qualidade")
)

df_invalidos = df_validacao.filter(F.col("flag_qualidade") == "ERRO")

total_validos = df_silver.count()
total_invalidos = df_invalidos.count()

print(f"✅ Validação concluída: {total_validos} válidos | {total_invalidos} inválidos")

In [0]:
delta_table = DeltaTable.forName(spark, TABLE_TARGET_VALIDOS)

delta_table.alias("target").merge(
    df_silver.alias("source"),
    "target.cd_pesquisa = source.cd_pesquisa"
).whenMatchedUpdateAll(
).whenNotMatchedInsertAll(
).execute()

print(f"✅ MERGE concluído")

